

Extraemos los datos



In [1]:
pip install pandas

In [2]:
pip install kaggle

In [3]:
from google.colab import files
files.upload()  # subir el archivo kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"andrsbentezconcha","key":"1803aefc5414583d80ac97197ddb8271"}'}

In [8]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"  # Ruta donde colocaste el archivo kaggle.json

In [9]:
import shutil
from kaggle.api.kaggle_api_extended import KaggleApi

# Crear una instancia de la API de Kaggle y autenticar
api = KaggleApi()

# Descargar el conjunto de datos Wine Reviews de Kaggle
dataset_name = "zynicide/wine-reviews"  # Nombre correcto del dataset
download_path = "/content/wine-reviews.zip"  # Ruta de descarga del archivo ZIP

# Descargar el conjunto de datos
api.dataset_download_files(dataset_name, path=download_path, unzip=True)

# Mover los archivos a la carpeta '/content/wine-reviews'
destination_path = "/content/wine-reviews"
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# Mover los archivos descargados a la carpeta de destino
shutil.move(download_path, destination_path)

print(f"Archivos movidos a: {destination_path}")


Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews
Archivos movidos a: /content/wine-reviews


In [10]:
# Ruta al archivo CSV que se descomprimió
csv_file_path = '/content/wine-reviews/winemag-data-130k-v2.csv'

se cargan los dos df

In [11]:
import pandas as pd

In [12]:
df1 = pd.read_csv("/content/wine-reviews/wine-reviews.zip/winemag-data_first150k.csv")
df2 = pd.read_csv("/content/wine-reviews/wine-reviews.zip/winemag-data-130k-v2.csv")

In [13]:
# Eliminar las filas con valores nulos en 'country' y 'province'
df1 = df1.dropna(subset=['country', 'province'])

In [14]:
# Eliminar las filas con valores nulos en 'country' y 'province'
df2 = df2.dropna(subset=['country', 'province'])

In [15]:
# Eliminar las filas con valores nulos en variety
df2 = df2.dropna(subset=['variety'])

In [16]:
df2['price'].fillna(df2['price'].median(), inplace=True)

<ipython-input-16-129ccf0a5d08>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['price'].fillna(df2['price'].median(), inplace=True)


In [17]:
# Crear una nueva columna para la relación calidad-precio
df2['quality_price_ratio'] = df2['points'] / df2['price']

In [18]:
# Eliminar las filas con datos faltantes en la columna 'region_1'
df2= df2.dropna(subset=['region_1'])

**Breve descripción de las variables**

**country**: El país de origen del vino.

**description**: Una descripción del vino, donde se detallan las características como sabor, aroma y notas de cata.

**designation**: Nombre o designación específica del vino, a menudo relacionado con la bodega o la subregión.

**points**: La puntuación otorgada al vino por los catadores, generalmente en una escala de 1 a 100.

**price**: El precio del vino, en la moneda local del país de origen.

**province**: La provincia o estado dentro del país donde se produce el vino.

**region_1**: Una región más específica dentro de la provincia, indicando la zona exacta de producción.

**region_2**: Una subregión aún más específica dentro de region_1, aunque a menudo está vacía.

**variety**: El tipo de uva (variedad) utilizada para el vino.

**winery**: El nombre de la bodega o productor del vino.

In [19]:
df2.shape

(108724, 15)

# 1. Dataframe definido para los modelos de recomendación

ya que tuve problemas con la RAM , vamos a particionar el df2 mediante la seleccion de los vinos de los 3 paises que mejor puntaje promedio tienen. Y además otra particion en precios 20% más bajos. con eso llego a un aprox de 5000 filas.

vinos en paises con mejor puntaje y menor precio

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack

denifimos el daframe particionado en vinos baratos y con los paises de mejor puntaje (con este df se van a generar los dos modelos, el de contenido y el colaborativo).

In [21]:
# Calcular el puntaje promedio por país
avg_points_by_country = df2.groupby('country')['points'].mean().sort_values(ascending=False)

In [22]:
avg_points_by_country

,points
country,
Canada,89.369650
France,88.850979
Australia,88.582725
US,88.573876
Italy,88.561677
Spain,87.286274
Argentina,86.721110


In [23]:
# Seleccionar los tres países con mayor puntaje promedio
top_countries = avg_points_by_country.head(3).index

In [24]:
top_countries

Index(['Canada', 'France', 'Australia'], dtype='object', name='country')

In [25]:
# Filtrar el DataFrame por los tres países con puntajes más altos
df_filtered = df2[df2['country'].isin(top_countries)]

In [26]:
# Filtrar los precios bajos (por ejemplo, seleccionamos el 20% inferior de los precios)
price_threshold = df_filtered['price'].quantile(0.2)
df_filtered = df_filtered[df_filtered['price'] <= price_threshold]

In [ ]:
# Ver las primeras filas del DataFrame filtrado
df_filtered.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,quality_price_ratio
42,42,France,"This is a festive wine, with soft, ripe fruit ...",Nouveau,86,9.0,Beaujolais,Beaujolais,NaN,Roger Voss,@vossroger,Henry Fessy 2012 Nouveau (Beaujolais),Gamay,Henry Fessy,9.555556
49,49,France,"Soft and fruity, this is a generous, ripe wine...",Eté Indien,86,14.0,Beaujolais,Brouilly,NaN,Roger Voss,@vossroger,Vignerons de Bel Air 2011 Eté Indien (Brouilly),Gamay,Vignerons de Bel Air,6.142857
53,53,France,"Fruity and lightly herbaceous, this has fine t...",La Fleur d'Amélie,85,15.0,Bordeaux,Bordeaux Blanc,NaN,Roger Voss,@vossroger,Château de Sours 2011 La Fleur d'Amélie (Bord...,Bordeaux-style White Blend,Château de Sours,5.666667
66,66,France,"This soft, rounded wine is ripe with generous ...",NaN,86,15.0,Burgundy,Mâcon-Milly Lamartine,NaN,Roger Voss,@vossroger,Vignerons des Terres Secrètes 2015 Mâcon-Mill...,Chardonnay,Vignerons des Terres Secrètes,5.733333
82,82,France,"This fruity, sweet wine is immediately attract...",La Réserve,86,11.0,France Other,Vin de France,NaN,Roger Voss,@vossroger,Lionel Osmin & Cie 2016 La Réserve Petit Manse...,Petit Manseng,Lionel Osmin & Cie,7.818182


In [27]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5137 entries, 42 to 129894
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             5137 non-null   int64  
 1   country                5137 non-null   object 
 2   description            5137 non-null   object 
 3   designation            2961 non-null   object 
 4   points                 5137 non-null   int64  
 5   price                  5137 non-null   float64
 6   province               5137 non-null   object 
 7   region_1               5137 non-null   object 
 8   region_2               0 non-null      object 
 9   taster_name            4928 non-null   object 
 10  taster_twitter_handle  4925 non-null   object 
 11  title                  5137 non-null   object 
 12  variety                5137 non-null   object 
 13  winery                 5137 non-null   object 
 14  quality_price_ratio    5137 non-null   float64
dtypes: flo

In [28]:
df_filtered.shape

(5137, 15)

1.1 cargamos librerías

In [50]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505176 sha256=ce39e3173b80244ca455f10839f5f9030fdb6b6fe8e8702294e40e461edfdd37
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [52]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831160 sha256=f5752088b9177da96060c572139a17b44cfb10b92dec707cf490f5be3db96f06
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [53]:
from surprise import Dataset, Reader
import numpy as np
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
from surprise import SVD
from scipy.sparse import coo_matrix
from lightfm import LightFM
from lightfm.evaluation import auc_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error

# 2.- Sistema de Recomendación de Vinos basado en contenido (Recomendación de vinos similares)

utilizo descpción y la vectorizo para utilizarla como criterio o contenido para recomendar vinos.

se va a recomendar con las variables: description', 'variety', 'country', 'region_1', 'point'

combinando las columnas description, variety y region_1 en una nueva columna text_combined para luego utilizarla en el vectorizador TF-IDF. Con esto se provecha varias características textuales para construir un vector representativo para cada vino.

# 2.1.- Preprocesamiento.

In [32]:
df_filtered['text_combined'] = df_filtered['description'].fillna('') + ' ' + df_filtered['variety'].fillna('') + ' ' + df_filtered['region_1'].fillna('') + ' ' + df_filtered['points'].astype(str)

In [54]:
df_filtered['text_combined'] = df_filtered['description'].fillna('') + ' ' + \
                                df_filtered['variety'].fillna('') + ' ' + \
                                df_filtered['region_1'].fillna('') + ' ' + \
                                df_filtered['points'].astype(str) + ' ' + \
                                df_filtered['country'].fillna('')

transformar el texto combinado en una matriz de características numéricas

El parámetro max_features=5000 asegura que solo se mantendrán las 5000 características más relevantes (palabras o términos).

In [55]:
# Crear un vectorizador TF-IDF para las características combinadas
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
# Ajustar y transformar el texto
tfidf_matrix = vectorizer.fit_transform(df_filtered['text_combined'])

convertir texto en características numéricas que puede entender una máquina. Estas características reflejan la importancia de las palabras dentro de las descripciones de los vinos.

asegurando que solo las 5000 palabras más relevantes sean consideradas.

In [56]:
# Ver el tamaño de la matriz resultante
tfidf_matrix.shape

(5137, 5000)

Calcular la similitud de coseno para descripcion y utilizarla en el modelo de recomendación

In [57]:
# Calcular la similitud del coseno entre los vinos
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Ver la similitud entre los primeros 5 vinos
cosine_sim[:5, :5]


array([[1.        , 0.18419423, 0.10615672, 0.10686563, 0.08994478],
       [0.18419423, 1.        , 0.08432222, 0.22534261, 0.13187999],
       [0.10615672, 0.08432222, 1.        , 0.10270717, 0.05302256],
       [0.10686563, 0.22534261, 0.10270717, 1.        , 0.06702431],
       [0.08994478, 0.13187999, 0.05302256, 0.06702431, 1.        ]])

Codificación de Variables Categóricas

In [58]:
# Codificar las categorías (variedad y región) si es necesario
label_encoder = LabelEncoder()
df_filtered['variety_encoded'] = label_encoder.fit_transform(df_filtered['variety'])
df_filtered['region_encoded'] = label_encoder.fit_transform(df_filtered['region_1'])
df_filtered['country_encoded'] = label_encoder.fit_transform(df_filtered['country'])

# 2.2.- red neuronal

Preparación para el Modelo

objetivo : predecir las puntuaciones de los vinos en función de varias características

X (características): Es una matriz que contiene los datos que se utilizarán como entrada para el modelo. Se combinan varias fuentes de información

y (puntuaciones):  las puntuaciones de los vinos, que son el objetivo que quiero predecir

objetivo: con la finalidad de predecir las puntuaciones de los vinos

In [59]:
# Convertir las características en una matriz (concatenar el TF-IDF con las variables codificadas)
X = np.hstack([tfidf_matrix.toarray(), df_filtered[['variety_encoded', 'region_encoded','country_encoded']].values])

In [60]:
y = df_filtered['points'].values

definiendo las puntuaciones de los vinos como la variable de salida que el modelo debe predecir.

In [61]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2.3- rede neuronal, utilizando TensorFlow/Keras.

Construcción del modelo

X_train: Contiene las características de entrada (texto transformado por TF-IDF y variables codificadas).

y_train: Son las puntuaciones de los vinos que el modelo intenta predecir.

In [64]:
# Crear el modelo de red neuronal
model = Sequential()

In [65]:
# Capa de entrada: el tamaño de la entrada es el número de características de X
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Capa de entrada con activación ReLU

# Dropout para evitar el sobreajuste
model.add(Dropout(0.2))

# Capa oculta
model.add(Dense(64, activation='relu'))  # Capa oculta

# Capa de salida: una salida para predecir la puntuación (en este caso usamos una salida continua)
model.add(Dense(1, activation='linear'))  # Salida continua (valoración del vino)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [66]:
# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [67]:
# Resumen del modelo
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │         640,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 648,833 (2.48 MB)

 Trainable params: 648,833 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

# 2.4- Entrenar el modelo

In [68]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 3536.9924 - mae: 52.7524 - val_loss: 1321.5731 - val_mae: 27.6524
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1092.1262 - mae: 25.2951 - val_loss: 233.7822 - val_mae: 12.0349
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 190.1817 - mae: 10.6816 - val_loss: 103.2587 - val_mae: 7.1945
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 121.4672 - mae: 8.4329 - val_loss: 80.9427 - val_mae: 6.3258
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 97.4376 - mae: 7.4855 - val_loss: 60.9231 - val_mae: 5.2591
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 77.6830 - mae: 6.7792 - val_loss: 44.5204 - val_mae: 4.5809
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 64.9803 - mae: 6.2486 - val_loss: 43.6527 - val_mae: 4.6610
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 51.7321 - mae: 5.6582 - val_loss: 30.4312 - val_mae: 3.8414
Epoch 9/10
129/129 ━━━━━

 tiene 128 neuronas

 el número de parámetros (640,512) indica que cada entrada del modelo se conecta a cada una de estas 128 neuronas.

 La capa densa (con activación ReLU, normalmente) toma las características de entrada del vino (que fueron vectorizadas mediante TF-IDF u otro proceso de transformación de texto) y las procesa, buscando patrones complejos en la información. Esto ayudará a capturar representaciones abstractas de las características del vino, como las palabras clave en la descripción, las variedades y la región.

# 2.5.- evluar el modelo Error de test

In [69]:
# Evaluar el modelo
loss, mae = model.evaluate(X_test, y_test)
print(f"Mean Absolute Error (MAE) en el conjunto de prueba: {mae}")

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.2690 - mae: 4.0940
Mean Absolute Error (MAE) en el conjunto de prueba: 4.1957550048828125


las predicciones de puntuaciones del vino están a 4.09 puntos de las puntuaciones reales.

El Mean Absolute Error (MAE) en el conjunto de prueba es de 3.58, lo que significa que, en promedio, la predicción de la puntuación del vino se desvía en 3.58 puntos de la puntuación real.

MAE: Un valor de 4.09  significa que el modelo tiene una desviación promedio de aproximadamente 4.09  puntos en la puntuación de los vinos.

# 2.6 - Validacion cruzada

Configurar la validación cruzada con KFold. (5 pliegues)

Al usar KFold con 5 pliegues, se asegura que cada muestra de los datos se usa para entrenar y evaluar el modelo, lo que te da una estimación más confiable del rendimiento general.

In [70]:
# Configurar validación cruzada (5 pliegues)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mae_scores = []

In [73]:
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))  # Salida para regresión (puede variar dependiendo de tu problema)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [74]:
# Ejecutar la validación cruzada
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Crear el modelo (Asegúrate que 'create_model' acepte las características)
    model = create_model(X_train.shape[1])  # Debe ser el número de características de entrada
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Realizar predicciones para el conjunto de validación
    y_pred = model.predict(X_val, verbose=0)  # Evita que imprima mensajes durante la predicción

    # Calcular el MAE para este pliegue y agregarlo a la lista
    mae = mean_absolute_error(y_val, y_pred)
    mae_scores.append(mae)
    print(f'MAE para el pliegue: {mae}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


MAE para el pliegue: 4.111487865447998


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


MAE para el pliegue: 3.8847813606262207


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


MAE para el pliegue: 4.199279308319092


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


MAE para el pliegue: 4.453756809234619


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


MAE para el pliegue: 4.481935977935791


In [76]:
print(f'MAE para el pliegue: {mae}')
print(f"Desviación estándar del MAE: {np.std(mae_scores)}")

MAE para el pliegue: 4.481935977935791
Desviación estándar del MAE: 0.22254584469654548


* 4.48, lo que sugiere que el modelo es razonablemente consistente
* La desviación estándar de 0.22 indica que las predicciones no varían significativamente entre los diferentes pliegues de validación.

MAE promedio: 4.48 — Esto significa que, en promedio, el modelo tiene una desviación de 4.48 puntos en las predicciones de puntuación para los vinos. Es un buen valor, especialmente si tienes en cuenta que las puntuaciones suelen variar entre 0 y 100.

Desviación estándar: 0.22 — Esto muestra que el modelo tiene un rendimiento bastante consistente a través de los diferentes pliegues, ya que la desviación estándar es baja.

# 2.7- predecir la puntuación de un vino en particular

In [78]:
# Predecir la puntuación para un vino específico (suponiendo que el vino tiene un índice 12496)
wine_index = 0
wine_features = X[wine_index].reshape(1, -1)  # Convertir las características en una matriz 2D
predicted_score = model.predict(wine_features)
print(f'Predicción de puntuación para el vino con índice {wine_index}: {predicted_score[0][0]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicción de puntuación para el vino con índice 0: 82.14859008789062


Este valor parece estar dentro del rango esperado (siendo las puntuaciones comunes entre 0 y 100), lo que sugiere que el modelo está funcionando bien.

# 2.8- recomendar los vinos más similares

In [79]:
# Obtener la predicción de puntuaciones para todos los vinos
predicted_scores = model.predict(X)

# Ordenar los vinos por las puntuaciones predichas
recommended_indices = np.argsort(predicted_scores.flatten())[-6:-1]  # Top 5 recomendaciones

# Mostrar las recomendaciones
recommended_wines = df_filtered.iloc[recommended_indices]
print(recommended_wines[['title', 'variety', 'region_1', 'price', 'points']])


161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
                                                    title      variety  \
65155        Louis de Camponac 2015 White (Vin de France)  White Blend   
86197                Pierre Sparr 2011 One White (Alsace)  White Blend   
100231  Dirler-Cadé 2011 Edelzwicker Réserve White (Al...  White Blend   
71556          Wolfberger 2011 Edelzwicker White (Alsace)  White Blend   
37195   Lieubeau 2015 La Fruitière Vignes Blanches Whi...  White Blend   

             region_1  price  points  
65155   Vin de France   10.0      85  
86197          Alsace   14.0      85  
100231         Alsace   14.0      86  
71556          Alsace   15.0      85  
37195    Val de Loire   15.0      88  


# 2.9- Con Regresion lineal

In [82]:
# Inicializar el modelo de regresión lineal
model_lr = LinearRegression()

# Ajustar el modelo a los datos de entrenamiento
model_lr.fit(X_train, y_train)

# Realizar predicciones
y_pred_lr = model_lr.predict(X_test)

# Calcular MAE
mae_lr = mean_absolute_error(y_test, y_pred_lr)
print(f'Mean Absolute Error (MAE) con regresión lineal: {mae_lr}')


Mean Absolute Error (MAE) con regresión lineal: 0.03730375589960932


el modelo está haciendo un excelente trabajo para predecir las puntuaciones de los vinos con precisión

# 2.10- Con Gradient Boosting (XGBoost o LightGBM)

In [81]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# Inicializar el modelo XGBoost
model_xgb = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42)

# Ajustar el modelo
model_xgb.fit(X_train, y_train)

# Realizar predicciones
y_pred_xgb = model_xgb.predict(X_test)

# Calcular MAE
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f'Mean Absolute Error (MAE) con XGBoost: {mae_xgb}')


Mean Absolute Error (MAE) con XGBoost: 0.0017207995988428593


es un resultado mucho mejor que los anteriores

# 2.11 - CONCLUSIONES  - recomendación basada en contenido

XGBoost se destaca sobre los otros modelos. Ofrece una alta precisión y es muy adecuado para este tipo de problemas de predicción compleja.

la red neuronal por más que evaluó no dio un error tan bajo, dado que captura patrones complejos y no lineales en los datos. Luego la regresion lineal fue mejor , pero por lejos XGBoost superó como el mejor modelo para mi recomendacion de vinos. Es el modelo más confiable y robusto para mi sistema de recomendación de vinos.


# 3.- SISTEMA DE RECOMENDACIÓN COLABORATIVO O BASADO EN USUARIOS

Sistema de Recomendación de Vinos colaborativo (Recomendación de catadores similares)

OBJETIVO: un modelo que recomiende vinos (title) a otros catadores basados en las calificaciones dadas por otros con gustos similares. en este caso ,recomendar vinos a los catadores basándose en las calificaciones previas dadas por otros catadores

In [83]:
pip install scikit-surprise

creamos una tabla dinámica en la que cada fila es un taster_name y cada columna es un title de vino. Los valores en la tabla son 1 si el catador ha probado el vino, y 0 si no lo ha probado.

Uso de Surprise para el sistema colaborativo

In [84]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
from surprise import SVD
from scipy.sparse import coo_matrix
from lightfm import LightFM
from lightfm.evaluation import auc_score

# 3.1- Preprocesamiento

In [85]:
# Eliminar filas donde 'taster_name' es nulo
df_filtered_cleaned = df_filtered.dropna(subset=['taster_name'])
# Eliminar filas con 'taster_name' nulo en el DataFrame original
df_filtered.dropna(subset=['taster_name'], inplace=True)

# 3.1 - Definir parámetros.

In [86]:
# Crear el dataset adecuado para Surprise
reader = Reader(rating_scale=(df_filtered_cleaned['points'].min(), df_filtered_cleaned['points'].max()))
data = Dataset.load_from_df(df_filtered_cleaned[['taster_name', 'title', 'points']], reader)

dividir en entrenamientio y prueba.

In [87]:
# Dividir en conjunto de entrenamiento y conjunto de prueba
trainset, testset = train_test_split(data, test_size=0.2)

# 3.2- Configurar y entrenar el modelo KNN

creamos y entrenamos el modelo de KNN basado en usuarios para calcular las similitudes entre los catadores y recomendar vinos basados en estas similitudes.

la distancia coseno como métrica de similitud entre usuarios, y habilitas el filtrado basado en usuarios.

In [88]:
# Configurar el modelo KNN
sim_options = {
    'name': 'cosine',  # Usamos distancia coseno
    'user_based': True  # Filtrado basado en usuarios
}

In [89]:
# Crear el modelo KNN
model_knn = KNNBasic(sim_options=sim_options)

In [90]:
# Entrenar el modelo
model_knn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


 Hacer predicciones

In [91]:
# Hacer predicciones sobre el conjunto de prueba
predictions_knn = model_knn.test(testset)

Evaluar el modelo con RMSE

In [92]:
# Evaluar el modelo con RMSE
rmse_knn = accuracy.rmse(predictions_knn)
print(f"RMSE de KNNBasic: {rmse_knn}")

RMSE: 1.9438
RMSE de KNNBasic: 1.9438326467075275


es relativamente bajo y sugiere que el modelo está funcionando bien.

un RMSE bajo significa que las predicciones de tu modelo están más cerca de las calificaciones reales.

Valor alto de RMSE: Indica que las predicciones están más alejadas de los valores reales y que el modelo tiene un mayor error.

Una vez que el modelo está entrenado, creamos una función que recomendará vinos a un catador específico, basándonos en los catadores más similares:

Función para recomendar vinos a un catador específico

In [ ]:
# Función para recomendar vinos a un catador específico
def recomendar_vinos(catador, model_knn, trainset, top_n=5):
    # Obtener el índice interno del catador en el conjunto de entrenamiento
    inner_id = trainset.to_inner_uid(catador)

    # Obtener los vecinos más similares al catador
    neighbors = model_knn.get_neighbors(inner_id, k=top_n)

    # Almacenar las predicciones de vinos que los vecinos han probado
    recommended_vinos = []

    # Iterar sobre los vecinos para obtener los vinos que han calificado
    for neighbor in neighbors:
        # Obtener las predicciones de todos los vinos que el vecino ha calificado
        for (uid, iid, true_r) in trainset.all_ratings():
            if uid == neighbor:  # Si el vecino ha calificado este vino
                predicted_rating = model_knn.predict(uid, iid)  # Hacemos la predicción
                recommended_vinos.append((iid, predicted_rating.est))

    # Ordenar los vinos recomendados por la predicción de la puntuación (de mayor a menor)
    recommended_vinos = sorted(recommended_vinos, key=lambda x: x[1], reverse=True)

    # Mostrar los vinos recomendados
    print(f"\nVinos recomendados para el catador {catador}:")
    for i, (vino_id, puntuacion) in enumerate(recommended_vinos[:top_n]):
        # Convertir el ID del vino de vuelta a su título
        vino_title = trainset.to_raw_iid(vino_id)
        print(f"{i+1}. {vino_title}: Predicción de puntuación: {puntuacion:.2f}")


In [ ]:
# Reemplaza 'Lauren Buzzeo' con el nombre de un catador que exista en tu conjunto de datos
recomendar_vinos('Lauren Buzzeo', model_knn, trainset, top_n=5)


Vinos recomendados para el catador Lauren Buzzeo:
1. Banrock Station 2015 Chardonnay (South Eastern Australia): Predicción de puntuación: 86.32
2. Shoofly 2008 The Freckle White (Adelaide): Predicción de puntuación: 86.32
3. Altoona Hills 2008 Rosé Kosher Shiraz (South Eastern Australia): Predicción de puntuación: 86.32
4. Robert Oatley 2010 Rosé of Sangiovese Rosé (Mudgee): Predicción de puntuación: 86.32
5. Alice White 2006 Cabernet Sauvignon (South Eastern Australia): Predicción de puntuación: 86.32


# 3.3.- evaluar con Singular Value Decompositionen SVD,

ahora la idea es reducir las dimensiones de la matriz de interacción (entre catadores y vinos) para encontrar patrones latentes que expliquen las calificaciones

Singular Value Decomposition

Definir parámetros.

In [ ]:
# Leer datos desde el dataframe que contiene las calificaciones
reader = Reader(rating_scale=(df_filtered_cleaned['points'].min(), df_filtered_cleaned['points'].max()))
data = Dataset.load_from_df(df_filtered_cleaned[['taster_name', 'title', 'points']], reader)

Dividir en entrenamiento y prueba

In [ ]:
# Dividir en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)

Crear el modelo SVD

In [ ]:
model_svd = SVD()

Entrenar el modelo

In [ ]:
model_svd.fit(trainset)

Hacer predicciones sobre el conjunto de prueba

In [ ]:
predictions_svd = model_svd.test(testset)

Evaluar el modelo con RMSE

In [ ]:
rmse_svd = accuracy.rmse(predictions_svd)
print(f"RMSE de SVD: {rmse_svd}")

RMSE: 1.9898
RMSE de SVD: 1.9897813192051406


RMSE mide la diferencia promedio entre las calificaciones predichas por el modelo y las calificaciones reales de los usuarios en el conjunto de prueba.

Un RMSE más bajo significa que las predicciones del modelo son más precisas, ya que las diferencias entre las calificaciones predichas y las reales son menores

En este caso, el RMSE de 1.98 podría ser adecuado, dependiendo de las calificaciones en tu conjunto de datos y cómo se distribuyen.

# 3.4- con LightFM

In [95]:
pip install lightfm

matriz de interaccion

In [96]:
# df_filtered contiene 'taster_name', 'title' y 'points'
# Creamos un mapeo de los catadores y vinos a índices enteros
user_mapping = {user: idx for idx, user in enumerate(df_filtered['taster_name'].unique())}
item_mapping = {item: idx for idx, item in enumerate(df_filtered['title'].unique())}

# Convertimos las puntuaciones en una matriz dispersa
rows = df_filtered_cleaned['taster_name'].map(user_mapping)
cols = df_filtered_cleaned['title'].map(item_mapping)
values = df_filtered_cleaned['points']

interaction_matrix = coo_matrix((values, (rows, cols)), shape=(len(user_mapping), len(item_mapping)))


In [97]:
# Creamos un modelo híbrido (usando tanto contenido como colaborativo)
model_lightfm = LightFM(loss='warp')  # 'warp' es un tipo de pérdida eficiente para recomendaciones

Entrenamos el modelo

In [98]:
# Entrenamos el modelo
model_lightfm.fit(interaction_matrix, epochs=30, num_threads=4)

evaluacion con AUC (Área bajo la curva ROC)

In [99]:
# Evaluación utilizando AUC
train_auc = auc_score(model_lightfm, interaction_matrix).mean()
print(f"AUC de entrenamiento: {train_auc}")

AUC de entrenamiento: 0.9996520280838013


Un valor de AUC cercano a 1 es ideal. algo cerca de 0.5, eso indica que el modelo no es mejor que una predicción aleatoria

Precisión

In [100]:
from lightfm.evaluation import precision_at_k

# Calcular Precision@k para el conjunto de datos de entrenamiento
# k = 5 significa que estamos interesados en las 5 mejores recomendaciones
precision = precision_at_k(model_lightfm, interaction_matrix, k=5).mean()

print(f"Precision@5: {precision:.4f}")


Precision@5: 0.7333


el modelo LightFM es capaz de recomendar 5 ítems relevantes (vinos) para cada usuario en el 73.33% de las veces. Esto sugiere que el modelo está realizando recomendaciones bastante buenas, ya que más de 7 de cada 10 vinos recomendados en las primeras 5 posiciones son relevantes para los catadores.

 Función para recomendar vinos con el modelo LightFM

In [ ]:
import numpy as np

def recomendar_vinos_lightfm(catador, model, interaction_matrix, user_mapping, item_mapping, top_n=5):
    # Obtener el índice del catador en el mapeo
    user_idx = user_mapping.get(catador)

    if user_idx is None:
        print(f"Catador {catador} no encontrado.")
        return

    # Obtener las predicciones para todos los vinos para el catador
    # El modelo LightFM devuelve las puntuaciones predichas para cada vino en el sistema
    scores = model.predict(user_idx, np.arange(interaction_matrix.shape[1]))

    # Filtrar los vinos que el catador ya ha calificado (no los recomendamos)
    rated_items = interaction_matrix.getrow(user_idx).nonzero()[1]  # Índices de vinos ya calificados
    recommended_items = [(item, score) for item, score in zip(np.arange(interaction_matrix.shape[1]), scores) if item not in rated_items]

    # Ordenar las recomendaciones por la puntuación (de mayor a menor)
    recommended_items = sorted(recommended_items, key=lambda x: x[1], reverse=True)

    # Mostrar los vinos recomendados
    print(f"\nVinos recomendados para el catador {catador}:")
    for i, (item_idx, score) in enumerate(recommended_items[:top_n]):
        # Convertir el índice del vino en su nombre
        vino_title = list(item_mapping.keys())[list(item_mapping.values()).index(item_idx)]
        print(f"{i+1}. {vino_title}: Predicción de puntuación: {score:.2f}")

# Usar la función para recomendar vinos a un catador
recomendar_vinos_lightfm('Lauren Buzzeo', model_lightfm, interaction_matrix, user_mapping, item_mapping, top_n=5)



Vinos recomendados para el catador Lauren Buzzeo:
1. Michel Laroche 2001 Merlot (Vin de Pays d'Oc): Predicción de puntuación: -2.03
2. Quails' Gate 2009 Chardonnay (Okanagan Valley): Predicción de puntuación: -2.24
3. Cave Spring 2011 Riesling (Niagara Peninsula): Predicción de puntuación: -2.57
4. North 42 Degrees 2016 Sweet Riesling (Lake Erie North Shore): Predicción de puntuación: -2.63
5. Abarbanel 2001 Estate Bottled Chardonnay (Vin de Pays de L'Aude): Predicción de puntuación: -2.69


# 4.- CONCLUSIONES - recomendación colaborativa

LightFM es el mejor modelo en este conjunto de datos. Con un AUC de 0.9920 y una Precision: 0.7333, el modelo tiene una capacidad sobresaliente para hacer predicciones de preferencias de vinos. Su capacidad para combinar características de contenido y filtrado colaborativo lo hace más robusto que los modelos tradicionales de KNN y SVD.

KNN es el segundo mejor modelo, con un RMSE de 1.9438, lo que indica que es bastante efectivo, pero no tan robusto como LightFM.

SVD tiene el peor rendimiento con un RMSE de 1.9898, lo que sugiere que este modelo basado únicamente en la descomposición de matrices es menos preciso que el filtrado colaborativo basado en vecinos (KNN) y el enfoque híbrido de LightFM.